In this assignment, you will use a pre-trained convnet to produce features for a classifier that can detect a single object type. This notebook has some code to help you get started. 

In [33]:
from imutils import paths
import pandas as pd
import requests
import cv2
import os
from os import listdir
from os.path import isfile, join
import os.path as osp
from tqdm import tqdm_notebook as tqdm
from google_images_download import google_images_download

img_folder = 'downloads'

In [13]:
def build_arguments(word):
    args = {}
    args['keywords'] = word
    args['limit'] = 100
    args['format'] = 'png'
    args['usage_rights'] = 'labeled-for-nocommercial-reuse'
    return args

response = google_images_download.googleimagesdownload()

### Run the following two cells

- change the positive and negative words
- running each cell downloads 100 images into a `downloads` folder with a subfolder of your word
- `word` can be any string or search criteria

In [64]:
# positive examples

word = 'red'
absolute_image_paths = response.download(build_arguments(word))

In [65]:
# negative examples

word = 'random'
absolute_image_paths = response.download(build_arguments(word))

## 1.) Run the following cell

* This imports needed Keras libraries
* Then, it gets the trained VGG19 imagenet model
* Then, it prints out the names of all the layers in that model

In [14]:
import numpy as np
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model

base_model = VGG19(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

input_3
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


### 2.) Determine your output layer

- try `predictions` first

In [19]:
layer = 'predictions'

model = Model(input=base_model.input, output=base_model.get_layer(layer).output)

/home/casey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  This is separate from the ipykernel package so we can avoid doing imports until


### Run the following cell

In [26]:
def get_image(img_path, xs=224,ys=224):
    x = image.load_img(img_path, target_size=(xs, ys))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    return x

def get_img_features(model, img):
    img = preprocess_input(img)
    yhat = model.predict(img)
    return yhat

def get_image_features(word):
    files = [f for f in listdir(osp.join(img_folder, word))] # grab all of the images in the folder
    image_vectors = []
    for f in tqdm(files):
        img = get_image(osp.join(img_folder, word, f)) 
        x_feats = get_img_features(model, img).flatten() # get features for each image
        image_vectors.append(x_feats) 
    return np.array(image_vectors)

## 3.) Evaluate a classifier for your `word`

* Using the positive and negative output from `base_model`, train a classifier (it can be a linear classifier from scikit-learn, if you'd like, but I would recommend the Keras Dense network we built for the previous assignment). 
* You'll need to split your data into Train and Test (I would recommend using half of the data for training, half for testing; you may opt for downloading more positive and negative examples)

In [27]:
pos_images = get_image_features('red') # get positive image vectors
neg_images = get_image_features('random') # get negative image vectors

/home/casey/anaconda3/lib/python3.6/site-packages/PIL/Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


### Prepare the data. Split to train/test sets

### Define model, train

### Evaluate

## 4.) Try a Different `base_model` 

* Repeat steps 3 and 4 above, only this time use a [pre-trained model other than VGG19](https://keras.io/applications/)
* (Answer in a markdown cell): Which model+layer works the best for this data? Why do you think that is?